In [7]:
import pandas as pd

beatmap_df = pd.read_csv("/mnt/L-HDD/dataset/beatmaps.csv")
beatmap_ids = set(beatmap_df["ID"].str.split("-").str[0])
beatmap_ids

{'1844020',
 '2246377',
 '909759',
 '2260345',
 '915063',
 '1058293',
 '886193',
 '1590068',
 '2262069',
 '850663',
 '2193011',
 '2238749',
 '2062538',
 '1121997',
 '518596',
 '44072',
 '2008892',
 '718454',
 '6892',
 '1920362',
 '925047',
 '1037306',
 '2228966',
 '288752',
 '2219255',
 '2228001',
 '925343',
 '658811',
 '41686',
 '1017271',
 '1744864',
 '2007899',
 '1765365',
 '590023',
 '1096826',
 '357539',
 '1032959',
 '36346',
 '458584',
 '2149007',
 '2244147',
 '1646146',
 '2274226',
 '1035167',
 '2224016',
 '2287975',
 '1002716',
 '2256868',
 '2230360',
 '1152932',
 '917614',
 '2269930',
 '846599',
 '37242',
 '730303',
 '901188',
 '2118959',
 '50324',
 '1850935',
 '11090',
 '595163',
 '1765424',
 '961477',
 '23078',
 '2280680',
 '2271578',
 '2224972',
 '2183696',
 '2298154',
 '351630',
 '12709',
 '25931',
 '2231608',
 '1264800',
 '2254381',
 '2270474',
 '956155',
 '2243803',
 '50638',
 '36849',
 '539042',
 '69118',
 '659614',
 '731933',
 '1205697',
 '1135593',
 '2249618',
 '20570

In [13]:
import os
import shutil

audio_path = "/mnt/L-HDD/dataset/audio/"
song_folders = os.listdir(audio_path)
removed = []
for folder in song_folders:
    if(folder not in beatmap_ids):
        removed.append(folder)
        shutil.rmtree(os.path.join(audio_path, folder))
removed

[]

In [14]:
import subprocess
from tqdm import tqdm

def is_audio_corrupted(file_path):
    try:
        result = subprocess.run(
            ["ffmpeg", "-v", "error", "-i", file_path, "-f", "null", "-"],
            stderr=subprocess.PIPE,
            stdout=subprocess.DEVNULL
        )
        if result.stderr:
            print(f"Corruption detected in {file_path}:\n{result.stderr.decode()}")
            return True
        return False
    except FileNotFoundError:
        return True


corrupted = []
for folder in tqdm(song_folders, desc="Checking audio files"):
    filename = os.listdir(os.path.join(audio_path, folder))[0]
    file_path = os.path.join(os.path.join(audio_path, folder), filename)
    if(is_audio_corrupted(file_path)):
        ids.append(file_path)
corrupted

Checking audio files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1926/1926 [08:01<00:00,  4.00it/s]


[]

In [5]:
def clear_fixed_files(directory):
    for file in os.listdir(directory):
        if file.startswith("fixed."):
            file_path = os.path.join(directory, file)
            os.remove(file_path)

def fix_bom_issue(file_path):
    parent = parent_folder = os.path.dirname(file_path)
    subprocess.run(["ffmpeg", "-i", file_path, "-c", "copy", os.path.join(parent, "fixed")], check=True)


def fix_header_issue(file_path):
    parent = parent_folder = os.path.dirname(file_path)
    subprocess.run(
            ["ffmpeg", "-i", file_path, "-acodec", "libmp3lame", "-b:a", "192k", os.path.join(parent, "fixed.mp3")],
            check=True
        )

In [8]:
for beatmap_path in tqdm(corrupted):
    parent_folder = os.path.dirname(beatmap_path)
    clear_fixed_files(parent_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 1219.33it/s]


In [9]:
for beatmap_path in tqdm(corrupted, desc="Fixing Header issue"):
    try:
        fix_header_issue(beatmap_path)
    except:
        continue

ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers                                                                                                                           | 0/42 [00:00<?, ?it/s]
  built with gcc 14.2.1 (GCC) 20240910
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enabl

In [10]:
for beatmap_path in tqdm(corrupted, desc="Fixing Bom issue"):
    try:
        fix_bom_issue(beatmap_path)
    except:
        continue

ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers                                                                                                                           | 0/42 [00:00<?, ?it/s]
  built with gcc 14.2.1 (GCC) 20240910
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enabl

In [13]:
def clear_corrupted_files(directory):
    files = os.listdir(directory)
    if(len(files) > 1):
        for file in files:
            if not file.startswith("fixed."):
                file_path = os.path.join(directory, file)
                os.remove(file_path)
            
for beatmap_path in tqdm(corrupted):
    parent_folder = os.path.dirname(beatmap_path)
    clear_corrupted_files(parent_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 1948.19it/s]


In [14]:
audio_path = "/mnt/L-HDD/dataset/audio/" 
song_folders = os.listdir(audio_path)
ids = []
for folder in tqdm(song_folders, desc="Checking audio files"):
    filename = os.listdir(os.path.join(audio_path, folder))[0]
    file_path = os.path.join(os.path.join(audio_path, folder), filename)
    if(is_audio_corrupted(file_path)):
        ids.append(file_path)
ids

Checking audio files:  96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊     | 1908/1978 [08:50<00:30,  2.29it/s]

Corruption detected in /mnt/L-HDD/dataset/audio/1810985/fixed.mp3:
[mp3 @ 0x618315eec680] Failed to find two consecutive MPEG audio frames.
[in#0 @ 0x618315eec3c0] Error opening input: Invalid data found when processing input
Error opening input file /mnt/L-HDD/dataset/audio/1810985/fixed.mp3.
Error opening input files: Invalid data found when processing input



Checking audio files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1978/1978 [09:08<00:00,  3.61it/s]


['/mnt/L-HDD/dataset/audio/1810985/fixed.mp3']

In [21]:
beatmap_ids = [id.split("/")[-2] for id in ids]
beatmap_ids

['1810985']

In [22]:
import shutil
for beatmap_id in beatmap_ids:
    shutil.rmtree(os.path.join("/mnt/L-HDD/dataset/audio/", beatmap_id))

In [25]:
import os
import pandas as pd

def remove_rows_by_ids(database_path, ids_to_remove):
    beatmaps_csv = os.path.join(database_path, "beatmaps.csv")
    hit_objects_csv = os.path.join(database_path, "hit_objects.csv")
    time_points_csv = os.path.join(database_path, "timing_points.csv")
    
    beatmaps_df = pd.read_csv(beatmaps_csv)
    hit_objects_df = pd.read_csv(hit_objects_csv)
    time_points_df = pd.read_csv(time_points_csv)

    beatmaps_df['beatmap_id'] = beatmaps_df['ID'].str.split("-").str[0]
    hit_objects_df['beatmap_id'] = hit_objects_df['ID'].str.split("-").str[0]
    time_points_df['beatmap_id'] = time_points_df['ID'].str.split("-").str[0]

    beatmaps_df = beatmaps_df[~beatmaps_df['beatmap_id'].isin(ids_to_remove)]
    hit_objects_df = hit_objects_df[~hit_objects_df['beatmap_id'].isin(ids_to_remove)]
    time_points_df = time_points_df[~time_points_df['beatmap_id'].isin(ids_to_remove)]

    beatmaps_df.to_csv(beatmaps_csv, index=False)
    hit_objects_df.to_csv(hit_objects_csv, index=False)
    time_points_df.to_csv(time_points_csv, index=False)

    print(f"Removed rows with IDs {ids_to_remove} from the CSV files.")

remove_rows_by_ids("/mnt/L-HDD/dataset/", beatmap_ids)


Removed rows with IDs ['1810985'] from the CSV files.
